In [ ]:
from crpropa import *

In [18]:
from pylab import *

# Synthetic turbulent field with CRPropa

### Properties of grid and field

In [ ]:
# Grid
spacing = 10 * pc
N = 100
# Field
Brms = 3 * nG
lmin = 50 * pc 
lmax = 1000 * pc 
index = 5./3. 
random_seed = 42

# Creation of turbulent field
spectrum = SimpleTurbulenceSpectrum(Brms, lmin, lmax, index)
grid = GridProperties(Vector3d(0), N, spacing)
BField = SimpleGridTurbulence(spectrum, grid, random_seed)

# Save field
dumpGrid(BField.getGrid(), 'exapmple_field.dat')

### Check of field properties

In [ ]:
print('Lc = {:.1f} kpc'.format(BField.getCorrelationLength() / pc))  
print('sqrt(<B^2>) = {:.1f} nG'.format(BField.getBrms() / nG))   
print('<|B|> = {:.1f} nG'.format(BField.getMeanFieldStrength() / nG))

### Load field

In [ ]:
BField = loadGrid(vgrid, 'example_field.dat')

### Evaluation of field components along the three axes

In [33]:
Bx = 0; By = 0; Bz = 0
for i in range(0,100):
    for j in range(0,100):
        for k in range(0,100):
            x = spacing*i
            y = spacing*j
            z = spacing*k
            field = BBField.getField(Vector3d(x,y,z) * pc) / nG
            Bx = Bx + field[0]
            By = By + field[0]
            Bz = Bz + field[0]

B = np.sqrt(Bx**2 + By**2 + Bz**2)
print('Total Bx', Bx)
print('Total By', By)
print('Total Bz', Bz)
print('Total B', B) # Corroboration for a zero-mean-field in grid

1000000

### Plot of field components (wrong, correcting...)

In [ ]:
plt.figure(figsize=(10, 7))
# x-axis
plt.subplot(2, 2, 1)
plt.plot(x, Bx_x, label=r'$B_x$')
plt.plot(x, By_x, label=r'$B_y$')
plt.plot(x, Bz_x, label=r'$B_z$')
plt.xscale('log')
plt.grid()
plt.xlabel('x [pc]', fontsize=17)
plt.ylabel('B [nG]', fontsize=17)
plt.yticks(fontsize=17)
plt.xticks(fontsize=17)
plt.title('Along x-axis', fontsize=17)
# y-axis
plt.subplot(2, 2, 2)
plt.plot(y, Bx_y, label=r'$B_x$')
plt.plot(y, By_y, label=r'$B_y$')
plt.plot(y, Bz_y, label=r'$B_z$')
plt.xscale('log')
plt.grid()
plt.xlabel('y [pc]', fontsize=17)
plt.ylabel('B [nG]', fontsize=17)
plt.yticks(fontsize=17)
plt.xticks(fontsize=17)
plt.title('Along y-axis', fontsize=17)
# z-axis
plt.subplot(2, 2, 3)
plt.plot(z, Bx_z, label=r'$B_x$')
plt.plot(z, By_z, label=r'$B_y$')
plt.plot(z, Bz_z, label=r'$B_z$')
plt.xscale('log')
plt.grid()
plt.xlabel('z [pc]', fontsize=17)
plt.ylabel('B [nG]', fontsize=17)
plt.yticks(fontsize=17)
plt.xticks(fontsize=17)
plt.title('Along z-axis', fontsize=17)
plt.savefig('Field_components.pdf')

## Uniform field $\vec{B_0}=B_0\hat{z}$ with turbulence $\vec{\delta B}=(\delta B_x,\delta B_y,0)$

## $\delta B_x=\delta B\cos(kz)$ and $\delta B_y=\pm\delta B\sin(kz)$ such that $\frac{\delta B_y}{\delta B_x}=\pm\tan(kz)$

### Computation of k-scales and power spectrum F(k)

In [ ]:
k = [0] * 601
F_k = [0] * 601
for i in range(0,601):
    z = 10**(i / 100)
    Bx, By, Bz = BBField.getField(Vector3d(0,0,z) * pc) / nG
    k[i] = (By / Bx)*(1 / z) 
    delta_B = np.cos(k*z) / Bx
    #delta_B = np.sin(k*z) / By
    F_k[i] = ( (delta_B**2) / (Brms**2) ) * k

### Plot of power spectrum

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(k, F_k)
plt.title('Power spectrum of turbulent field', fontsize=17)
plt.xlabel('k [1/pc]', fontsize=17)
plt.ylabel('F(k)', fontsize=17)
plt.grid()
plt.yticks(fontsize=17)
plt.xticks(fontsize=17)
plt.savefig('Spectrum_synthetic_field.pdf')